In [1]:
#!pip install --upgrade celery

In [2]:
#import celery
#print(celery.__version__)
#5.1.2

In [3]:
!sudo rabbitmqctl add_user myguest myguestpwd
!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Error: user_already_exists: myguest
Setting permissions for user "myguest" in vhost "/" ...


In [4]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


This code will **run on a server machine**. It will ask worker machines to complete some tasks and send the results back to the server.

In the same folder with this code there is a test.py file 
You will need to run that code on a **worker** machine with <br>

**"celery -A test worker --loglevel=info"**

Then that machine becomes a worker and will be able to run the "test" application (i.e. in this case the echo function in test.py) whenever the broker requests it.

The echo function will be called by these commands on the remote (=worker) machines:

from test import echo
res = echo.delay('Python rocks!'); print(type(res)); print(res)
res.result

In [5]:
from test import echo

Before you run the next cell, you will need to <br>
"pip install celery" on the worker machine and then run <br>
"celery -A test worker --loglevel=info" on that machine.



In [6]:
# We will run the echo function on the worker machine.
# The fun.delay(arg) function sends the arg message to the worker's fun function and run it on the worker machine.
# For more info, see https://docs.celeryproject.org/en/latest/userguide/calling.html
    
# This code will send 'Python rocks 1 :) !' to the worker machine, and put its response into res1
res1 = echo.delay('Python rocks 1 :) !')

print(type(res1))
print(res1)

<class 'celery.result.AsyncResult'>
ea637743-fdce-4adb-ba0f-81d0e83aa0bc


In [7]:
# You will need backend to be able to get the below results.
# As you can see in the test.py we are using 'rpc://...' backend.
res1.ready()

True

In [8]:
res1.result

'Response from worker: Python rocks 1 :) !'

In [9]:
# Let us repeat it with a different message: 
# Tasks will be distributed round robin among the workers

res2 = echo.delay('Python rocks 2 :) !'); print(type(res2)); print(res2)
res2.ready()

<class 'celery.result.AsyncResult'>
fdd93b60-5d5f-48ef-9734-d7029dc995b4


False

As you can see res2 is asyncronous and might not be immedialtely available.

In [10]:
print(res2.result)

Response from worker: Python rocks 2 :) !


In [11]:
res2.ready()

True

In [12]:
echo('hello')

'Response from worker: hello'